In [1]:
from lxml import etree

def strip_namespaces(tree):
    for elem in tree.iter():
        if '}' in elem.tag:
            elem.tag = elem.tag.split('}', 1)[1]
    return tree

# Load and clean GEXF
tree = etree.parse("road_graph_with_intersections.gexf")
tree = strip_namespaces(tree)
tree.write("cleaned.gexf", encoding="utf-8", xml_declaration=True)

print("Cleaned GEXF saved.")

Cleaned GEXF saved.


In [2]:
import networkx as nx

G = nx.read_gexf("cleaned.gexf")
print(f"Loaded graph with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")

Loaded graph with 23421 nodes and 31948 edges.


In [7]:
import networkx as nx
from neo4j import GraphDatabase
from tqdm import tqdm

# === Config ===
gexf_path = "cleaned.gexf"
uri = "bolt://localhost:7687"
user = "neo4j"
password = "neo4j_password"  # change this if needed

# === Load graph ===
print("Loading GEXF...")
G = nx.read_gexf(gexf_path)
print(f"Graph loaded: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

# === Neo4j connection ===
driver = GraphDatabase.driver(uri, auth=(user, password))

def create_node(tx, node_id, props):
    tx.run("MERGE (n:Intersection {id: $id}) SET n += $props", id=node_id, props=props)

def create_edge(tx, source, target, props):
    tx.run("""
        MATCH (a:Intersection {id: $source}), (b:Intersection {id: $target})
        MERGE (a)-[r:ROAD_TO]->(b)
        SET r += $props
    """, source=source, target=target, props=props)

with driver.session() as session:
    print("Pushing nodes...")
    for node, attrs in tqdm(G.nodes(data=True)):
        session.execute_write(create_node, str(node), dict(attrs))

    print("Pushing edges (with reverse)...")
    for u, v, attrs in tqdm(G.edges(data=True)):
        edge_data = dict(attrs)
        session.execute_write(create_edge, str(u), str(v), edge_data)
        session.execute_write(create_edge, str(v), str(u), edge_data)  # reverse edge

driver.close()
print("✅ Upload complete with bidirectional edges.")

Loading GEXF...
Graph loaded: 23421 nodes, 31948 edges
Pushing nodes...


100%|███████████████████████████████████████████████████████████████████████████| 23421/23421 [02:52<00:00, 135.73it/s]


Pushing edges (with reverse)...


100%|████████████████████████████████████████████████████████████████████████████| 31948/31948 [13:13<00:00, 40.26it/s]

✅ Upload complete with bidirectional edges.
